In [1]:
!git clone https://github.com/sameerkhanna786/CVUEBA.git

Cloning into 'CVUEBA'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), done.


In [2]:
ls

CVUEBA/  sample_data/


In [3]:
cd CVUEBA

/content/CVUEBA


In [4]:
ls

CVUEBA.py               nondynamic.pkl      sae_hopt.py       train.csv
dataset.csv             prep_data_model.py  SAE.hyperopt      valid.csv
exec_aug.py             README.md           SAE.py
extract_non_dynamic.py  requirements.txt    split_dataset.py
generate_images.py      run.sh              test.csv


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!bash run.sh

串流輸出內容已截斷至最後 5000 行。
1/1 [==============================] - 0s 15ms/step
100% 74/74 [02:29<00:00,  2.02s/it]
1/1 [==============================] - 0s 16ms/step
100% 145/145 [04:16<00:00,  1.77s/it]
Traceback (most recent call last):
  File "exec_aug.py", line 84, in <module>
    shutil.copytree(f"{IN_FOLDER}/0", f"{OUT_FOLDER}/0", dirs_exist_ok=True)
TypeError: copytree() got an unexpected keyword argument 'dirs_exist_ok'
Traceback (most recent call last):
  File "CVUEBA.py", line 4, in <module>
    from tensorflow.python.keras.applications.resnet import ResNet50
ModuleNotFoundError: No module named 'tensorflow.python.keras.applications'


In [26]:
cd ..

/content/CVUEBA


In [27]:
import numpy as np
import cv2
import random
import shutil
import os
from tqdm import tqdm

"""
Novel augmentations proposed in the paper.
For demonstration purposes, we only apply
each augmentation once per attack encoding.
"""

IN_FOLDER = "TrainImages"
OUT_FOLDER = "AugImages"

def channel_swap(path, in_folder, out_folder):
	im = cv2.imread(path)
	channel_1 = im[:, :, 0]
	channel_2 = im[:, :, 1]
	channel_3 = im[:, :, 2]
	stack = np.stack((channel_2, channel_1, channel_3), axis=-1)

	file_name = path.split(".")[0]
	file_type = path.split(".")[1]
	write_name = f"{file_name}_swap.{file_type}"
	write_name = write_name.replace(in_folder, out_folder)
	cv2.imwrite(write_name, stack)

def get_user_from_path(path):
	file = path.split("/")[2]
	return file.split("_")[0]

def get_random_benign_from_user(user):
	folder = os.path.join(IN_FOLDER, "0")
	files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
	userfiles = [f for f in files if user in f]
	if len(userfiles) == 0:
		return None
	choice_file = random.choice(userfiles)
	return os.path.join(folder, choice_file)

def channel_replace(path, in_folder, out_folder):
	im = cv2.imread(path)
	channel_3 = im[:, :, 2]

	im_user = get_user_from_path(path)
	rand_path = get_random_benign_from_user(im_user)
	if rand_path is None:
		return
	rand_im = cv2.imread(rand_path)
	channel_1 = rand_im[:, :, 0]
	channel_2 = rand_im[:, :, 1]

	stack = np.stack((channel_1, channel_2, channel_3), axis=-1)

	file_name = path.split(".")[0]
	file_type = path.split(".")[1]
	write_name = f"{file_name}_replace.{file_type}"
	write_name = write_name.replace(in_folder, out_folder)
	cv2.imwrite(write_name, stack)

try:
	os.makedirs(f"{OUT_FOLDER}/0")
except:
	pass

try:
	os.makedirs(f"{OUT_FOLDER}/1")
except:
	pass

try:
	os.makedirs(f"{OUT_FOLDER}/2")
except:
	pass

try:
	os.makedirs(f"{OUT_FOLDER}/3")
except:
	pass

# Do not augment benign images. Just copy those over
#shutil.copytree(f"{IN_FOLDER}/0", f"{OUT_FOLDER}/0")
#shutil.copytree(f"{IN_FOLDER}/0", f"{OUT_FOLDER}/0", dirs_exist_ok=True)
for scenario in [1, 2, 3]:
	directory_path = os.path.join(IN_FOLDER, str(scenario))
	images = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
	for image in tqdm(images):
		path = os.path.join(directory_path, image)
		channel_swap(path, IN_FOLDER, OUT_FOLDER)
		channel_replace(path, IN_FOLDER, OUT_FOLDER)

100%|██████████| 2/2 [00:00<00:00,  9.57it/s]


In [19]:
ls

In [7]:
!python3 CVUEBA.py

Traceback (most recent call last):
  File "CVUEBA.py", line 4, in <module>
    from tensorflow.python.keras.applications.resnet import ResNet50
ModuleNotFoundError: No module named 'tensorflow.python.keras.applications'


In [28]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
#from tensorflow.python.keras.applications.resnet import ResNet50
#from tensorflow.python.keras.models import Sequential, Model
#from tensorflow.python.keras.layers import *
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
import numpy as np
import ssl
from sklearn.metrics import *
from prep_data_model import *

"""
Load or Train dual-input classifier.
Use the data loading API defined in prep_data_model
to extract data
"""

# Need to download ResNet weight (trained on ImageNet)
# This circumvents potential download issues of weights.
ssl._create_default_https_context = ssl._create_unverified_context

print("Loading data...")
#images, non_behave, y = load_data("AugImages")
images, non_behave, y = load_data("TestImages")
y[y != 0] = 1

print("...Done")

res_model = ResNet50(include_top = False, pooling = "avg", input_shape=(32, 32, 3))
for i, layer in enumerate(res_model.layers):
	if i <= 170:
		layer.trainable = False
	print(f"Layer {i} Out: {layer} Trainable: {layer.trainable}")

try:
	model = tf.keras.models.load_model("cvueba")
	print("Finished Loading Model!")
except:
	print("Model not found. Generating and training!")

	image_input = Input(shape=(32, 32, 3))
	vector_input = Input(shape=(47,))
	res_out = res_model(image_input)

	concat_layer = Concatenate()([vector_input, res_out])

	out = Dense(1024, activation='relu', kernel_regularizer="l2")(concat_layer)
	out = Dense(512, activation='relu', kernel_regularizer="l2")(out)
	out = Dense(256, activation='relu', kernel_regularizer="l2")(out)
	out = Dense(1, activation='sigmoid')(out)

	model = Model(inputs=[image_input, vector_input], outputs=out)

	METRICS = [
		tf.keras.metrics.BinaryAccuracy(name='accuracy'),
		tf.keras.metrics.Precision(name='precision'),
		tf.keras.metrics.Recall(name='recall'),
	]

	model.compile(loss='binary_crossentropy', optimizer="adam", metrics=METRICS)
	model.fit([images, non_behave], y, batch_size=128, epochs=100, shuffle=True)

	model.save("cvueba")

test_images, test_non_behave, y_test = load_data("TestImages")
y_test[y_test!=0] = 1

print("Predicting...")
y_pred = model.predict([test_images, test_non_behave])
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1
print("...Finished!")

bal_accuracy = balanced_accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Balanced Accuracy: {bal_accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Loading data...
...Done
94765736/94765736 [==============================] - 1s 0us/step
Layer 0 Out: <keras.engine.input_layer.InputLayer object at 0x7f90ef87c310> Trainable: False
Layer 1 Out: <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f90e20c1290> Trainable: False
Layer 2 Out: <keras.layers.convolutional.conv2d.Conv2D object at 0x7f90e2067410> Trainable: False
Layer 3 Out: <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f90e2067510> Trainable: False
Layer 4 Out: <keras.layers.core.activation.Activation object at 0x7f90e178ba10> Trainable: False
Layer 5 Out: <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f90e17f5950> Trainable: False
Layer 6 Out: <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f90e1818f10> Trainable: False
Layer 7 Out: <keras.layers.convolutional.conv2d.Conv2D object at 0x7f90e1818a10> Trainable: False
Layer 8 Out: <keras.layers.normalization.batch_normalization.BatchNormalization

Predicting...
130/130 [==============================] - 21s 152ms/step
...Finished!
Balanced Accuracy: 1.0
F1 Score: 1.0
Precision: 1.0
Recall: 1.0
